In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog
import numpy as np
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import itertools
import threading
import os
import joblib  # Add this import for joblib support
import warnings
warnings.filterwarnings("ignore", message=".*does not have valid feature names.*")

# Change the maping 
# Change the input and target variables 
# Change the Limits for combination

# Define a separate global variable for the contour plot
fig_contour = None  # New variable to store the contour plot
cancel_generation = False
cancel_check = False

# Map slider feature labels to model training feature names
feature_name_mapping = {
    "Level-A-Launder (m)": "Level-A-Launder",
    "Level-P-Launder (m)": "Level-P-Launder",
    "Level-BP-Launder (m)": "Level-BP-Launder",
    "Level-WetWell1-1 (m)": "Level-WetWell1-1",
    "Level-WetWell1-2 (m)": "Level-WetWell1-2",
    "Level-WetWell2-1 (m)": "Level-WetWell2-1",
    "Level-WetWell2-2 (m)": "Level-WetWell2-2",
    "Pressure-OceanDischarge (kPa)": "Pressure-OceanDicharge",
    "Level-S1 (m)": "Level-S1",
    "Level-S2 (m)": "Level-S2",
    "Level-S3 (m)": "Level-S3",
    "Time (h)": "Hour",

}


# Create main application window
root = tk.Tk()
root.title("Enhanced Wastewater Aeration Process Simulation")
root.geometry("1200x900")
root.configure(bg="#f0f0f0")

# Create notebook for tabs
notebook = ttk.Notebook(root)
notebook.pack(padx=10, pady=10, expand=True)

# Create main tab for sliders and predicted features
main_frame = ttk.Frame(notebook)
notebook.add(main_frame, text="Main Interface")

# Function to browse and load the model
def load_selected_model():
    model_path = filedialog.askopenfilename(filetypes=[("Model Files", "*.h5 *.pkl *.joblib")])
    if model_path:
        model_type_display = (
            "H5 Model" if model_path.endswith(".h5")
            else "PKL Model" if model_path.endswith(".pkl")
            else "Joblib Model" if model_path.endswith(".joblib")
            else "Unknown Model"
        )
        model_selector_var.set(model_type_display)

        global multi_output_model, model_type, target_menu, target_var
        try:
            if model_path.endswith(".h5"):
                multi_output_model = load_model(model_path, compile=False)
                model_type = "h5"
            elif model_path.endswith(".pkl"):
                with open(model_path, 'rb') as f:
                    multi_output_model = pickle.load(f)
                model_type = "pkl"
            elif model_path.endswith(".joblib"):
                multi_output_model = joblib.load(model_path)
                model_type = "joblib"
            else:
                print("Unsupported file type.")
                return

            # Recreate target menu if destroyed
            target_var = tk.StringVar()
            target_menu = ttk.OptionMenu(plot_frame, target_var, "", *textbox_labels)
            target_menu.grid(row=len(feature_names) + 1, column=0, padx=10, pady=10)
            update_target_menu(textbox_labels)

            predict_values()  # Update textboxes immediately after loading the model

        except Exception as e:
            print(f"Error loading model: {e}")
            model_selector_var.set("Select Model")


  


# Dropdown for selecting the model
model_selector_var = tk.StringVar(value="Select Model")
model_selector_button = ttk.Button(main_frame, text="Browse Model", command=load_selected_model)
model_selector_button.grid(row=0, column=0, padx=10, pady=10, sticky="w")
tk.Label(main_frame, textvariable=model_selector_var, bg="#f0f0f0", font=("Arial", 10)).grid(row=0, column=1, padx=10, pady=10, sticky="w")

# Slider features
slider_labels = [
    ("Level-A-Launder (m)", 0, 10, 0.01, 5.95),
    ("Level-P-Launder (m)", 0, 10, 0.01, 6.81),
    ("Level-BP-Launder (m)", 0, 10, 0.01, 2.09),
    ("Level-WetWell1-1 (m)", 0, 10, 0.1, 3),
    ("Level-WetWell1-2 (m)", 0, 10, 0.1, 3),
    ("Level-WetWell2-1 (m)", 0, 10, 0.1, 3),
    ("Level-WetWell2-2 (m)", 0, 10, 0.1, 3),
    ("Pressure-OceanDischarge (kPa)", 0, 1000, 10, 87),
    ("Level-S1 (m)", 0, 5, 0.1, 5),
    ("Level-S2 (m)", 0, 5, 0.1, 5),
    ("Level-S3 (m)", 0, 5, 0.1, 5),
    ("Time (h)", 0, 24, 1, 11)

]

slider_vars = []
slider_value_labels = []

# Create sliders with labels
for i, (label, min_val, max_val, step, default) in enumerate(slider_labels):
    tk.Label(main_frame, text=label, bg="#f0f0f0", font=("Arial", 10)).grid(row=i+1, column=0, padx=10, pady=5, sticky="w")
    slider_var = tk.DoubleVar(value=default)
    slider = ttk.Scale(
        main_frame, from_=min_val, to=max_val, orient="horizontal",
        variable=slider_var,
        length=300,
        command=lambda e, var=slider_var, lbl=i: update_label(var, slider_value_labels[lbl])
    )
    slider.grid(row=i+1, column=1, columnspan=4, padx=10, pady=5, sticky="we")
    value_label = tk.Label(main_frame, text=f"{default}", bg="#f0f0f0", font=("Arial", 10))
    value_label.grid(row=i+1, column=5, padx=10, pady=5)
    slider_value_labels.append(value_label)
    slider_vars.append(slider_var)

# Predicted features
textbox_labels = [
    "Flow-FEPS (L/s)"
]
textboxes = []

# Create textboxes for predictions
for i, label in enumerate(textbox_labels):
    tk.Label(main_frame, text=label, bg="#f0f0f0", font=("Arial", 10), anchor="center").grid(
        row=len(slider_labels) + 2 + (i // 6), column=(i % 6), padx=5, pady=5, sticky="we"
    )
    text = ttk.Entry(main_frame, font=("Arial", 10), width=15, justify="center")
    text.grid(row=len(slider_labels) + 3 + (i // 6), column=(i % 6), padx=5, pady=5, sticky="we")
    textboxes.append(text)

# Function to predict values based on the selected model
def predict_values():
    if not model_selector_var.get().startswith("Select"):
        values = [var.get() for var in slider_vars]
        values_array = np.array([values])

        try:
            # Handle different model types
            if model_type == "h5":
                predictions = multi_output_model.predict(values_array)[0]  # First output if multi-dimensional
            elif model_type in ["pkl", "joblib"]:
                # Create a DataFrame for scikit-learn models
                input_df = pd.DataFrame(values_array)
                predictions = multi_output_model.predict(input_df)

            # Ensure predictions are iterable
            predictions = np.ravel(predictions)  # Flatten to 1D if multi-dimensional

            # Update textboxes with predictions
            for i, prediction in enumerate(predictions):
                textboxes[i].delete(0, tk.END)
                textboxes[i].insert(0, f"{float(prediction):.2f}")  # Convert to float before formatting

        except Exception as e:
            print(f"Error during prediction: {e}")




# Function to update slider labels dynamically
def update_label(var, label):
    label.config(text=f"{var.get():.1f}")
    predict_values()

# Predict values on startup
predict_values()

# Create tab for 3D Surface Plot
plot_frame = ttk.Frame(notebook)
notebook.add(plot_frame, text="3D Surface Plot")

# Checkboxes for feature selection
feature_names = [label[0] for label in slider_labels]
feature_vars = {name: tk.BooleanVar() for name in feature_names}

for idx, name in enumerate(feature_names):
    tk.Checkbutton(plot_frame, text=name, variable=feature_vars[name]).grid(row=idx, column=0, sticky="w", padx=5, pady=2)

# Dropdown for target selection
def update_target_menu(labels):
    target_menu['menu'].delete(0, 'end')
    for label in labels:
        target_menu['menu'].add_command(label=label, command=lambda value=label: target_var.set(value))
    if labels:
        target_var.set(labels[0])

target_var = tk.StringVar()
target_menu = ttk.OptionMenu(plot_frame, target_var, "", *textbox_labels)
target_menu.grid(row=len(feature_names) + 1, column=0, padx=10, pady=10)
update_target_menu(textbox_labels)

# Function to save the 3D plot and its data
def save_3d_plot():
    file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    if file_path:
        fig.savefig(file_path, dpi=600, transparent=True)
        print(f"3D plot saved to {file_path}")

        data_file_path = file_path.replace(".png", "_data.csv")
        data = {
            "X": X.ravel(),
            "Y": Y.ravel(),
            "Z": Z.ravel()
        }
        pd.DataFrame(data).to_csv(data_file_path, index=False)
        print(f"3D plot data saved to {data_file_path}")
        
# Function to save the contour plot and its data
def save_contour_plot():
    global fig_contour  # Ensure we're using the contour figure
    
    if fig_contour is None:
        print("No contour plot available to save.")
        return

    file_path = filedialog.asksaveasfilename(
        defaultextension=".png", 
        filetypes=[("PNG files", "*.png"), ("All files", "*.*")]
    )

    if file_path:
        # Save the figure with tight layout to remove excess margins
        fig_contour.savefig(file_path, dpi=600, transparent=True, bbox_inches='tight', pad_inches=0.1)
        print(f"Contour plot saved to {file_path} with 600 DPI and reduced margins")

        # Save the contour data as well
        data_file_path = file_path.replace(".png", "_data.csv")
        data = {
            "X": X.ravel(),
            "Y": Y.ravel(),
            "Z": Z.ravel()
        }
        pd.DataFrame(data).to_csv(data_file_path, index=False)
        print(f"Contour plot data saved to {data_file_path}")





def plot_3d_surface():
    global target_menu, target_var

    # Validate model selection
    if model_selector_var.get().startswith("Select"):
        print("Please select a model before plotting.")
        root.after(0, lambda: status_label.config(text="Please select a model before plotting."))
        return

    # Validate feature selection
    selected_features = [name for name, var in feature_vars.items() if var.get()]
    if len(selected_features) != 2:
        print("Please select exactly two features.")
        root.after(0, lambda: status_label.config(text="Please select exactly two features for plotting."))
        return

    # Validate target selection
    target = target_var.get()
    if not target:
        print("Please select a target feature.")
        root.after(0, lambda: status_label.config(text="Please select a target feature for plotting."))
        return

    # Clear the plot frame to avoid duplicate widgets
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Recreate the target menu
    target_var = tk.StringVar()
    target_menu = ttk.OptionMenu(plot_frame, target_var, "", *textbox_labels)
    target_menu.grid(row=len(feature_names) + 1, column=0, padx=10, pady=10)
    target_var.set(textbox_labels[0])

    global X, Y, Z, fig
    x_idx = feature_names.index(selected_features[0])
    y_idx = feature_names.index(selected_features[1])
    target_idx = textbox_labels.index(target)

    x_vals = np.linspace(slider_labels[x_idx][1], slider_labels[x_idx][2], 50)
    y_vals = np.linspace(slider_labels[y_idx][1], slider_labels[y_idx][2], 50)
    X, Y = np.meshgrid(x_vals, y_vals)

    grid_shape = X.shape
    batch_input = np.zeros((grid_shape[0] * grid_shape[1], len(slider_vars)))

    for i, slider in enumerate(slider_vars):
        batch_input[:, i] = slider.get()

    batch_input[:, x_idx] = X.ravel()
    batch_input[:, y_idx] = Y.ravel()

    input_df = pd.DataFrame(batch_input)
    input_df.columns = [feature_name_mapping.get(col, col) for col in input_df.columns]

    try:
        if model_type == "h5":
            predictions = multi_output_model.predict(input_df.values)
        elif model_type in ["pkl", "joblib"]:
            predictions = multi_output_model.predict(input_df)
            
        # Ensure predictions are 2D for consistent handling
        if len(predictions.shape) == 1:  # If 1D, make it 2D
            predictions = np.expand_dims(predictions, axis=1)

        Z = predictions[:, target_idx].reshape(grid_shape)

        fig = Figure(figsize=(8, 6), dpi=100)
        ax = fig.add_subplot(111, projection="3d")
        surf = ax.plot_surface(X, Y, Z, cmap="viridis")

        # Set axes labels with styling
        ax.set_xlabel(selected_features[0], fontsize=14, fontweight='bold', fontname='Arial')
        ax.set_ylabel(selected_features[1], fontsize=14, fontweight='bold', fontname='Arial')
        ax.set_zlabel(target, fontsize=14, fontweight='bold', fontname='Arial')

        # Add colorbar with proper formatting
        fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, pad=0.1)

        canvas = FigureCanvasTkAgg(fig, plot_frame)
        canvas.get_tk_widget().grid(row=0, column=1, rowspan=20, padx=10, pady=10, sticky="nsew")
        canvas.draw()

    except Exception as e:
        print(f"Error during plotting: {e}")

    # Re-add feature selection checkboxes
    for idx, name in enumerate(feature_names):
        tk.Checkbutton(plot_frame, text=name, variable=feature_vars[name]).grid(row=idx, column=0, sticky="w", padx=5, pady=2)

    # Recreate buttons with proper placement
    plot_button = ttk.Button(plot_frame, text="Plot 3D Surface", command=plot_3d_surface)
    plot_button.grid(row=len(feature_names) + 2, column=0, pady=10, sticky="w")

    heatmap_button = ttk.Button(plot_frame, text="Plot Heatmap/Contour", command=plot_2d_heatmap_or_contour)
    heatmap_button.grid(row=len(feature_names) + 3, column=0, pady=10, sticky="w")

    save_button = ttk.Button(plot_frame, text="Save 3D Plot", command=save_3d_plot)
    save_button.grid(row=len(feature_names) + 4, column=0, padx=10, pady=10, sticky="w")







plot_button = ttk.Button(plot_frame, text="Plot 3D Surface", command=plot_3d_surface)
plot_button.grid(row=len(feature_names) + 2, column=0, pady=10, sticky="w")

# Create tab for Feature Combinations
combination_frame = ttk.Frame(notebook)
notebook.add(combination_frame, text="Feature Combinations")




def plot_2d_heatmap_or_contour():
    global target_menu, target_var, fig_contour

    if model_selector_var.get().startswith("Select"):
        print("Please select a model before plotting.")
        root.after(0, lambda: status_label.config(text="Please select a model before plotting."))
        return

    selected_features = [name for name, var in feature_vars.items() if var.get()]
    if len(selected_features) != 2:
        print("Please select exactly two features.")
        root.after(0, lambda: status_label.config(text="Please select exactly two features for plotting."))
        return

    target = target_var.get()
    if not target:
        print("Please select a target feature.")
        root.after(0, lambda: status_label.config(text="Please select a target feature for plotting."))
        return

    for widget in plot_frame.winfo_children():
        widget.destroy()

    target_var = tk.StringVar()
    target_menu = ttk.OptionMenu(plot_frame, target_var, "", *textbox_labels)
    target_menu.grid(row=len(feature_names) + 1, column=0, padx=10, pady=10)
    target_var.set(textbox_labels[0])

    x_idx = feature_names.index(selected_features[0])
    y_idx = feature_names.index(selected_features[1])
    target_idx = textbox_labels.index(target)

    x_vals = np.linspace(slider_labels[x_idx][1], slider_labels[x_idx][2], 50)
    y_vals = np.linspace(slider_labels[y_idx][1], slider_labels[y_idx][2], 50)
    X, Y = np.meshgrid(x_vals, y_vals)

    grid_shape = X.shape
    batch_input = np.zeros((grid_shape[0] * grid_shape[1], len(slider_vars)))

    for i, slider in enumerate(slider_vars):
        batch_input[:, i] = slider.get()

    batch_input[:, x_idx] = X.ravel()
    batch_input[:, y_idx] = Y.ravel()

    input_df = pd.DataFrame(batch_input)
    input_df.columns = [feature_name_mapping.get(col, col) for col in input_df.columns]

    try:
        if model_type == "h5":
            predictions = multi_output_model.predict(input_df.values)
        elif model_type in ["pkl", "joblib"]:
            predictions = multi_output_model.predict(input_df)

        if len(predictions.shape) == 1:
            predictions = np.expand_dims(predictions, axis=1)

        Z = predictions[:, target_idx].reshape(grid_shape)

        fig_contour = Figure(figsize=(8, 6), dpi=100)
        ax = fig_contour.add_subplot(111)

        # Plot heatmap
        heatmap = ax.imshow(
            Z, extent=[x_vals.min(), x_vals.max(), y_vals.min(), y_vals.max()],
            origin="lower", aspect="auto", cmap="viridis"
        )

        # Plot contour with proper formatting
        contour = ax.contour(X, Y, Z, colors="black", linewidths=0.8)
        
        # Format contour labels to be **integers** and use Arial font
        contour_labels = ax.clabel(contour, inline=True, fmt='%d', fontsize=14)  

        # Apply Arial font to contour labels
        for txt in contour_labels:
            txt.set_fontname("Arial")  
            txt.set_fontsize(14)

        # Set axis labels with Arial and bold style
        ax.set_xlabel(selected_features[0], fontsize=16, fontweight='bold', fontname='Arial')
        ax.set_ylabel(selected_features[1], fontsize=16, fontweight='bold', fontname='Arial')

        # Change tick labels to Arial and increase font size
        ax.tick_params(axis='both', labelsize=14)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname("Arial")
            label.set_fontsize(14)

        # Add colorbar with Arial font settings
        cbar = fig_contour.colorbar(heatmap, ax=ax, label=target)
        cbar.set_label(target, fontsize=16, fontweight='bold', fontname='Arial')
        cbar.ax.tick_params(labelsize=14)
        for label in cbar.ax.get_yticklabels():
            label.set_fontname("Arial")
            label.set_fontsize(14)

        canvas = FigureCanvasTkAgg(fig_contour, plot_frame)
        canvas.get_tk_widget().grid(row=0, column=1, rowspan=20, padx=10, pady=10, sticky="nsew")
        canvas.draw()

    except Exception as e:
        print(f"Error during plotting: {e}")




    
    # Re-add feature selection checkboxes
    for idx, name in enumerate(feature_names):
        tk.Checkbutton(plot_frame, text=name, variable=feature_vars[name]).grid(row=idx, column=0, sticky="w", padx=5, pady=2)

    # Recreate buttons with proper placement
    plot_button = ttk.Button(plot_frame, text="Plot 3D Surface", command=plot_3d_surface)
    plot_button.grid(row=len(feature_names) + 2, column=0, pady=10, sticky="w")

    heatmap_button = ttk.Button(plot_frame, text="Plot Heatmap/Contour", command=plot_2d_heatmap_or_contour)
    heatmap_button.grid(row=len(feature_names) + 3, column=0, pady=10, sticky="w")

    save_contour_button = ttk.Button(plot_frame, text="Save Contour Plot", command=save_contour_plot)
    save_contour_button.grid(row=len(feature_names) + 4, column=0, padx=10, pady=10, sticky="w")








def generate_combinations():
    def worker():
        global cancel_generation
        cancel_generation = False  # Reset the flag when the process starts

        # Use root.after to safely update the UI
        root.after(0, lambda: status_label.config(text="Generating combinations, please wait..."))

        combinations = []
        file_path = 'feature_combinations.csv'

        if os.path.exists(file_path):
            os.remove(file_path)

        for values in itertools.product(*[np.arange(label[1], label[2] + label[3], label[3]) for label in slider_labels]):
            if cancel_generation:  # Check if the process should stop
                root.after(0, lambda: status_label.config(text="Combination generation canceled."))
                return

            combinations.append(list(values))
            if len(combinations) >= 1000:
                df = pd.DataFrame(combinations, columns=[label[0] for label in slider_labels])
                df.columns = [feature_name_mapping.get(col, col) for col in df.columns]
                df.to_csv(file_path, mode='a', header=not os.path.exists(file_path), index=False)
                combinations = []

        if combinations:
            df = pd.DataFrame(combinations, columns=[label[0] for label in slider_labels])
            df.columns = [feature_name_mapping.get(col, col) for col in df.columns]
            df.to_csv(file_path, mode='a', header=not os.path.exists(file_path), index=False)

        root.after(0, lambda: status_label.config(text="Combinations generation completed."))

    # Run the worker in a separate thread
    threading.Thread(target=worker, daemon=True).start()






def check_combinations():
    global cancel_check
    cancel_check = False  # Reset the flag when the process starts

    def worker():
        root.after(0, lambda: status_label.config(text="Checking combinations, please wait...") if status_label.winfo_exists() else None)
        if model_selector_var.get().startswith("Select"):
            root.after(0, lambda: status_label.config(text="No model selected.") if status_label.winfo_exists() else None)
            return

        try:
            file_path = 'feature_combinations.csv'
            if not os.path.exists(file_path):
                root.after(0, lambda: status_label.config(text="Feature combinations file not found.") if status_label.winfo_exists() else None)
                return

            df = pd.read_csv(file_path)
            renamed_columns = [feature_name_mapping.get(col, col) for col in df.columns]
            df.columns = renamed_columns

            batch_size = 100000
            valid_combinations = []

           
            # Define individual conditions for each target
            target_conditions = [
                lambda preds: preds[0] > 500,                # Flow-FEPS

            ]
            
            # Define a final condition that ensures all target conditions are met
            final_condition = lambda preds: all(cond(preds) for cond in target_conditions)

            for start in range(0, len(df), batch_size):
                if cancel_check:  # Stop if the reset button was pressed
                    root.after(0, lambda: status_label.config(text="Combination check canceled.") if status_label.winfo_exists() else None)
                    return

                batch = df.iloc[start:start + batch_size]

                # Make predictions based on model type
                if model_type in ["pkl", "joblib"]:
                    predictions = multi_output_model.predict(batch)
                elif model_type == "h5":
                    predictions = multi_output_model.predict(batch.values)

                for i, prediction in enumerate(predictions):
                    if isinstance(prediction, np.ndarray):
                        prediction = prediction.flatten()
                
                    # Check the final condition
                    if final_condition(prediction):
                        valid_combinations.append(list(batch.iloc[i]) + list(prediction))



            if valid_combinations:
                columns = list(df.columns) + textbox_labels
                df_valid = pd.DataFrame(valid_combinations, columns=columns)
                root.after(0, lambda: display_results(df_valid, columns))
            else:
                root.after(0, lambda: status_label.config(text="No valid combinations found.") if status_label.winfo_exists() else None)

        except Exception as e:
            print(f"Error during combination check: {e}")
            root.after(0, lambda: status_label.config(text="Error occurred during combination check.") if status_label.winfo_exists() else None)

    threading.Thread(target=worker, daemon=True).start()


 


def display_results(df_valid, columns):
    # Limit displayed rows to 20
    displayed_data = df_valid.head(20)

    # Store the full data for saving
    global full_results
    full_results = df_valid

    # Clear existing widgets in the combination frame
    for widget in combination_frame.winfo_children():
        widget.grid_forget()

    # Create a Treeview table
    table = ttk.Treeview(combination_frame, columns=columns, show="headings", height=15)
    for col in columns:
        table.heading(col, text=col)
        table.column(col, width=100, anchor="center")  # Set a smaller column width

    table.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

    # Add scrollbar
    scrollbar = ttk.Scrollbar(combination_frame, orient="vertical", command=table.yview)
    table.configure(yscrollcommand=scrollbar.set)
    scrollbar.grid(row=0, column=1, sticky="ns")

    # Populate the table with the limited data
    for _, row in displayed_data.iterrows():
        table.insert("", "end", values=list(row))

    # Add the save button
    save_button = ttk.Button(combination_frame, text="Save Results", command=save_results)
    save_button.grid(row=1, column=0, padx=10, pady=10, sticky="w")

def save_results():
    if full_results is None or full_results.empty:
        print("No data to save.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
    if file_path:
        full_results.to_csv(file_path, index=False)
        print(f"Results saved to {file_path}")

def initialize_combination_tab():
    """Recreate the Feature Combinations tab's labels and buttons."""
    # Add labels and buttons back to the combination frame
    combination_button = ttk.Button(combination_frame, text="Generate Combinations", command=generate_combinations)
    combination_button.grid(row=0, column=0, padx=10, pady=10, sticky="w")

    check_button = ttk.Button(combination_frame, text="Check Combinations", command=check_combinations)
    check_button.grid(row=1, column=0, padx=10, pady=10, sticky="w")

    cancel_button = ttk.Button(combination_frame, text="Cancel Generating/Checking Combinations", command=lambda: cancel_process("generate"))
    cancel_button.grid(row=2, column=0, padx=10, pady=10, sticky="w")

    global status_label
    status_label = tk.Label(combination_frame, text="", bg="#f0f0f0", font=("Arial", 10), anchor="w")
    status_label.grid(row=3, column=0, padx=10, pady=10, sticky="w")

def cancel_process(process_type):
    """Cancel an ongoing process like generating or checking combinations."""
    global cancel_generation, cancel_check

    cancel_generation = True
    if status_label.winfo_exists():
        status_label.config(text="Combination generation canceled.")

    cancel_check = True
    if status_label.winfo_exists():
        status_label.config(text="Combination checking canceled.")


# Reset functionality ensures consistency
def reset_all():
    global cancel_generation, cancel_check, target_menu, target_var
    cancel_generation = True
    cancel_check = True  # Stop ongoing combination checking

    # Reset sliders to default values
    for i, (_, _, _, _, default) in enumerate(slider_labels):
        slider_vars[i].set(default)

    # Recalculate predictions
    predict_values()

    # Clear and reinitialize the 3D plot tab
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Recreate feature checkboxes
    for idx, name in enumerate(feature_names):
        tk.Checkbutton(plot_frame, text=name, variable=feature_vars[name]).grid(row=idx, column=0, sticky="w", padx=5, pady=2)

    # Recreate target menu
    target_var = tk.StringVar()
    target_menu = ttk.OptionMenu(plot_frame, target_var, "", *textbox_labels)
    target_menu.grid(row=len(feature_names) + 1, column=0, padx=10, pady=10)
    target_var.set(textbox_labels[0])

    # Add plot buttons back
    plot_button = ttk.Button(plot_frame, text="Plot 3D Surface", command=plot_3d_surface)
    plot_button.grid(row=len(feature_names) + 2, column=0, pady=10, sticky="w")

    heatmap_button = ttk.Button(plot_frame, text="Plot Heatmap/Contour", command=plot_2d_heatmap_or_contour)
    heatmap_button.grid(row=len(feature_names) + 3, column=0, pady=10, sticky="w")

    # Clear and reinitialize the combinations tab
    for widget in combination_frame.winfo_children():
        widget.destroy()
    initialize_combination_tab()

    print("All functionality has been reset.")

# Add the heatmap button to the plot frame
heatmap_button = ttk.Button(plot_frame, text="Plot Heatmap/Contour", command=plot_2d_heatmap_or_contour)
heatmap_button.grid(row=len(feature_names) + 3, column=0, pady=10, sticky="w")







# Add "Generate Combinations" Button
combination_button = ttk.Button(combination_frame, text="Generate Combinations", command=generate_combinations)
combination_button.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Add "Check Combinations" Button
check_button = ttk.Button(combination_frame, text="Check Combinations", command=check_combinations)
check_button.grid(row=1, column=0, padx=10, pady=10, sticky="w")

# Add "Cancel" Button
cancel_button = ttk.Button(combination_frame, text="Cancel Generating/Checking Combinations", command=lambda: cancel_process("generate"))
cancel_button.grid(row=2, column=0, padx=10, pady=10, sticky="w")

# Add the status label
status_label = tk.Label(combination_frame, text="", bg="#f0f0f0", font=("Arial", 10), anchor="w")
status_label.grid(row=3, column=0, padx=10, pady=10, sticky="w")

# Add Reset button to the Main Interface (top-right corner)
reset_button = ttk.Button(main_frame, text="Reset", command=reset_all)
reset_button.grid(row=0, column=6, padx=10, pady=10, sticky="e")




root.mainloop()